# Monthly Dominant Policies and Policy GDP & Unemployment Impact

## Find domininat policy for every for a region

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateparser import parse
from copy import deepcopy

In [2]:
import os
os.chdir("../")

In [17]:
from pandemic_functions.delphi_functions.DELPHI_model_policy_scenarios import read_oxford_country_policy_data, read_policy_data_us_only, get_dominant_policy
from pandemic_functions.pandemic_params import future_policies
from cost_functions.economic_cost.economic_data.economic_params import TOTAL_GDP

In [15]:
start_date = "2020-03-01"
end_date =  "2020-06-30"

region = 'ES'
country = 'Spain'
province = 'None'

if country == 'US':
    policy_data = read_policy_data_us_only(province, start_date=start_date, end_date=end_date)
else:
    policy_data = read_oxford_country_policy_data(country=country, start_date=start_date, end_date=end_date)

In [5]:
end_dt = parse(end_date)

dt1 = parse(start_date)
dt2 = dt1 + relativedelta(months=1, days=-1)

dominant_policy_dict = {"month_of": [], "dominant_policy": []}

while dt2 <= end_dt:
    dominant_policy_dict["month_of"].append(dt1.date())
    dominant_policy_dict["dominant_policy"].append(get_dominant_policy(policy_data, start_date=dt1, end_date=dt2))
    dt1 = dt2 + relativedelta(days=1)
    dt2 = dt1 + relativedelta(months=1, days=-1)

dominant_policy_df = pd.DataFrame.from_dict(dominant_policy_dict)

In [6]:
dominant_policy_df

,month_of,dominant_policy
0,2020-03-01,Lockdown
1,2020-04-01,Lockdown
2,2020-05-01,Lockdown
3,2020-06-01,Restrict_Mass_Gatherings_and_Schools_and_Others


In [7]:
from pandemic_functions.pandemic_params import default_dict_normalized_policy_gamma

## Calculating GDP Impact

In [8]:
gdp_impact = pd.read_csv("cost_functions/economic_cost/economic_data/gdp/ES.csv")

In [9]:
gdp_impact["date"]=[datetime(int(r.year), int(r.month), 1).date() for _, r in gdp_impact.iterrows()]

In [10]:
df = pd.merge(gdp_impact, dominant_policy_df, how="left", left_on="date", right_on="month_of")

In [11]:
df

,year,month,c,i,g,date,month_of,dominant_policy
0,2020,1,0.0,0.0,1.0,2020-01-01,NaN,NaN
1,2020,2,0.0,0.0,1.0,2020-02-01,NaN,NaN
2,2020,3,-9.9,-2.1,1.0,2020-03-01,2020-03-01,Lockdown
3,2020,4,-13.9,-5.1,0.8,2020-04-01,2020-04-01,Lockdown
4,2020,5,-13.9,-5.1,0.8,2020-05-01,2020-05-01,Lockdown
5,2020,6,-13.9,-5.1,0.8,2020-06-01,2020-06-01,Restrict_Mass_Gatherings_and_Schools_and_Others
6,2020,7,-4.5,-1.8,0.9,2020-07-01,NaN,NaN
7,2020,8,-4.5,-1.8,0.9,2020-08-01,NaN,NaN
8,2020,9,-4.5,-1.8,0.9,2020-09-01,NaN,NaN
9,2020,10,-6.8,-2.1,1.4,2020-10-01,NaN,NaN


In [12]:
df["GDP"] = df.c + df.i - df.g
df.dropna(subset=['dominant_policy'], inplace=True)

In [13]:
policy_gamma_df = pd.DataFrame.from_dict(default_dict_normalized_policy_gamma, orient='index')
policy_gamma_df.columns = ['gamma']
policy_gamma_df

,gamma
No_Measure,1.000
Restrict_Mass_Gatherings,0.873
Authorize_Schools_but_Restrict_Mass_Gatherings_and_Others,0.794
Mass_Gatherings_Authorized_But_Others_Restricted,0.668
Restrict_Mass_Gatherings_and_Schools,0.479
Restrict_Mass_Gatherings_and_Schools_and_Others,0.423
Lockdown,0.239


In [18]:
mean_GDP_impact = df.groupby("dominant_policy").agg({'c':'mean', 'i':'mean', 'g':'mean', 'GDP': 'mean'})/( TOTAL_GDP[region] / (12*1e9) )
mean_GDP_impact = mean_GDP_impact.join(policy_gamma_df, how='left')

In [15]:
mean_GDP_impact

,c,i,g,GDP,gamma
dominant_policy,,,,,
Lockdown,-0.085198,-0.027797,0.0,-0.112994,0.239
Restrict_Mass_Gatherings_and_Schools_and_Others,-0.094237,-0.034576,0.0,-0.128814,0.423


In [19]:
from sklearn.linear_model import LinearRegression

x = np.array(mean_GDP_impact.gamma.to_list())
x = 1-x # reversing gamma to make the line pass through 0 for No Measure
y = np.array(mean_GDP_impact.GDP.to_list())
ind = np.array(mean_GDP_impact.index.to_list())

In [20]:
model = LinearRegression(fit_intercept=False).fit(x.reshape(-1,1), y)
m = model.coef_[0]
m

-0.2617428543184304

In [21]:
policy_gamma_df["pred_gdp_impact"] = [m*(1-g) for g in policy_gamma_df["gamma"]]

In [22]:
policy_gamma_df

,gamma,pred_gdp_impact
No_Measure,1.000,-0.000000
Restrict_Mass_Gatherings,0.873,-0.033241
Authorize_Schools_but_Restrict_Mass_Gatherings_and_Others,0.794,-0.053919
Mass_Gatherings_Authorized_But_Others_Restricted,0.668,-0.086899
Restrict_Mass_Gatherings_and_Schools,0.479,-0.136368
Restrict_Mass_Gatherings_and_Schools_and_Others,0.423,-0.151026
Lockdown,0.239,-0.199186


## Calculating Employment Impact

In [20]:
emp_impact = pd.read_csv("cost_functions/economic_cost/economic_data/unemployment/ES.csv")

In [21]:
emp_impact["date"]=[datetime(int(r.year), int(r.month), 1).date() for _, r in emp_impact.iterrows()]

In [22]:
df = pd.merge(emp_impact, dominant_policy_df, how="left", left_on="date", right_on="month_of")

In [23]:
mean_emp_impact = df.groupby("dominant_policy").agg({'unemployment_gain':'mean'})
mean_emp_impact = mean_emp_impact.join(policy_gamma_df, how='left')

In [24]:
mean_emp_impact

,unemployment_gain,gamma,pred_gdp_impact
dominant_policy,,,
Lockdown,1.4,0.239,-0.133764
Restrict_Mass_Gatherings_and_Schools_and_Others,2.4,0.423,-0.101421


In [25]:
x = np.array(mean_emp_impact.gamma.to_list())
x = 1-x # reversing gamma to make the line pass through 0 for No Measure
y = np.array(mean_emp_impact.unemployment_gain.to_list())
ind = np.array(mean_emp_impact.index.to_list())

In [26]:
model_emp = LinearRegression(fit_intercept=False).fit(x.reshape(-1,1), y)
m_emp = model_emp.coef_[0]
m_emp

2.6864755221753196

In [27]:
policy_gamma_df["pred_unemployment_gain"] = [m_emp*(1-g) for g in policy_gamma_df["gamma"]]

In [28]:
policy_gamma_df

,gamma,pred_gdp_impact,pred_unemployment_gain
No_Measure,1.000,-0.000000,0.000000
Restrict_Mass_Gatherings,0.873,-0.022323,0.341182
Authorize_Schools_but_Restrict_Mass_Gatherings_and_Others,0.794,-0.036209,0.553414
Mass_Gatherings_Authorized_But_Others_Restricted,0.668,-0.058357,0.891910
Restrict_Mass_Gatherings_and_Schools,0.479,-0.091578,1.399654
Restrict_Mass_Gatherings_and_Schools_and_Others,0.423,-0.101421,1.550096
Lockdown,0.239,-0.133764,2.044408
